In [1]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline


In [2]:
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
from gensim.models import Word2Vec,Phrases

In [1]:
from tensorflow import keras 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Embedding,Dropout,Conv1D,MaxPool1D,GRU,LSTM,Dense,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,roc_curve

In [6]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abdo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
train_data=pd.read_csv("labeledTrainData.tsv",header=0,delimiter='\t')
unlabeled_data=pd.read_csv("unlabeledTrainData.tsv",error_bad_lines=False,delimiter='\t')
test_data=pd.read_csv("testData.tsv",header=0,delimiter='\t')

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [8]:
datasets=[train_data,unlabeled_data,test_data]
titles=['Train Data', 'Unlabeled Train Data', 'Test Data']
for dataset,title in zip(datasets,titles):
    print(title)
    #dataset.info()
    display(dataset.head())

Train Data


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


Unlabeled Train Data


,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


Test Data


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [9]:
from data import preprocess
REPLACE_WITH_SPACE =re.compile(r'[^A-Za-z\s]')
#stop_words=set(stopwords.words("english"))
#lemmatizer=WordNetLemmatizer()


In [10]:
all_reviews=np.array([],dtype=str)

for dataset in datasets:
    all_reviews=np.concatenate((all_reviews,dataset.review),axis=0)

print('Total number of review:',len(all_reviews))

all_reviews=np.array(list(map(lambda x: preprocess(x,len(all_reviews)),all_reviews)))

counter =0 
X_train_data = all_reviews[:train_data.shape[0]]
Y_train_data = train_data.sentiment.values

Total number of review: 99998


In [11]:
bigrams=Phrases(sentences=all_reviews)
trigrams=Phrases(sentences=bigrams[all_reviews])


In [12]:
print(bigrams['space station near the solar system new york'.split()])


['space_station', 'near', 'the', 'solar_system', 'new_york']


In [ ]:
#trigrams['hello']

In [13]:
embedding_vector_size=256
trigrams_model=Word2Vec(
            sentences=trigrams[bigrams[all_reviews]],
            size=embedding_vector_size,
            min_count=3,window=5,workers=4
            )
print("Vocabulary size:",len(trigrams_model.wv.vocab))

Vocabulary size: 91823


In [14]:
def vectorize_data(data, vocab:dict) -> list:
    keys = list(vocab.keys())
    filter_unknown = lambda word:vocab.get(word,None) is not None
    encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
    vectorized = list(map(encode, data))
    print('Vectorize sentences... (done)')
    return vectorized
    
print('Convert sentences to sentences with ngrams...', end='\r')
X_data = trigrams[bigrams[X_train_data]]
print('Convert sentences to sentences with ngrams... (done)')
MAX_REVIEW_LENGTH = 150
X_pad = pad_sequences(sequences=vectorize_data(X_data, vocab=trigrams_model.wv.vocab), maxlen=MAX_REVIEW_LENGTH, padding='post')
print('Transform sentences to sequences... (done)')

Convert sentences to sentences with ngrams... (done)
Vectorize sentences... (done)
Transform sentences to sequences... (done)


In [ ]:
from model import build_model

lstm_model= build_model(embedding_martix=trigrams_model.wv.vectors,input_length=MAX_REVIEW_LENGTH,use_lstm=True)
lstm_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])


X_train, X_test, y_train, y_test = train_test_split(X_pad, Y_train_data, test_size=0.05, shuffle=True, random_state=42)
history = lstm_model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=100, epochs=5)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 256)          23506688  
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               394240    
____________________________________________________________

In [ ]:
fig, (axis1, axis2) = plt.subplots(nrows=1, ncols=2, figsize=(16,6))

# summarize history for accuracy
axis1.plot(history.history['acc'], label='Train', linewidth=3)
axis1.plot(history.history['val_acc'], label='Validation', linewidth=3)
axis1.set_title('Model accuracy', fontsize=16)
axis1.set_ylabel('accuracy')
axis1.set_xlabel('epoch')
axis1.legend(loc='upper left')

# summarize history for loss
axis2.plot(history.history['loss'], label='Train', linewidth=3)
axis2.plot(history.history['val_loss'], label='Validation', linewidth=3)
axis2.set_title('Model loss', fontsize=16)
axis2.set_ylabel('loss')
axis2.set_xlabel('epoch')
axis2.legend(loc='upper right')
plt.show()

In [ ]:
gru_model = build_model(embedding_martix=trigrams_model.wv.vectors, input_length=MAX_REVIEW_LENGTH, use_lstm=False)
gru_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
gru_model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=100, epochs=5)


In [ ]:
y_test_pred_lstm = lstm_model.predict(X_test)
y_test_pred_gru = gru_model.predict(X_test)

print("The AUC socre for GRU model is : %.4f." %roc_auc_score(y_test, y_test_pred_gru))
print("The AUC socre for LSTM model is : %.4f." %roc_auc_score(y_test, y_test_pred_lstm))

y_pred_list = [y_test_pred_gru, y_test_pred_lstm]
label_list = ["GRU", "LSTM"]
pred_label = zip(y_pred_list, label_list)
for y_pred, lbl in pred_label:
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    plt.plot(fpr, tpr, label = lbl)

plt.xlabel("True Postive Rate")
plt.ylabel("False Positive Rate")
plt.title("ROC Curve for RNN Models")
plt.legend()
plt.show()